In [1]:
import numpy as np
import pandas as pd

## Regression Tree

In [2]:
df = pd.read_csv('/kaggle/input/chemical/train.csv')
df.head()

,Time,OZONE,NO2,temp,humidity,no2op1,no2op2,o3op1,o3op2
0,2019-03-27 13:01:00,77.59,6.881,36.2,38.9,199.0,200.0,240.0,197.0
1,2019-03-27 13:03:00,78.71,11.057,36.3,37.7,196.0,200.0,237.0,196.0
2,2019-03-27 13:04:00,78.85,8.596,36.7,38.0,195.0,199.0,235.0,196.0
3,2019-03-27 13:07:00,79.27,7.248,37.0,37.5,193.0,198.0,233.0,195.0
4,2019-03-27 13:08:00,80.01,8.638,36.8,36.8,191.0,198.0,231.0,195.0


In [3]:
X_train = df[["temp","humidity","no2op1","no2op2","o3op1","o3op2"]]
X_train.head()

,temp,humidity,no2op1,no2op2,o3op1,o3op2
0,36.2,38.9,199.0,200.0,240.0,197.0
1,36.3,37.7,196.0,200.0,237.0,196.0
2,36.7,38.0,195.0,199.0,235.0,196.0
3,37.0,37.5,193.0,198.0,233.0,195.0
4,36.8,36.8,191.0,198.0,231.0,195.0


In [4]:
y_train = df[["OZONE","NO2"]]
y_train.head()

,OZONE,NO2
0,77.59,6.881
1,78.71,11.057
2,78.85,8.596
3,79.27,7.248
4,80.01,8.638


In [5]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
max_depth = 30
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100,max_depth=max_depth, random_state=0)
)
regr_multirf.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=30,
                                                     random_state=0))

In [7]:
df_test = pd.read_csv('/kaggle/input/chemical-test/dummy_test.csv')
df_test.head()

,Time,OZONE,NO2,temp,humidity,no2op1,no2op2,o3op1,o3op2
0,2019-03-27 17:05:00,71.327,8.801,41.2,28.9,179.0,194.0,220.0,192.0
1,2019-03-27 17:06:00,72.317,5.536,41.1,28.8,181.0,196.0,222.0,192.0
2,2019-03-27 17:07:00,74.440,4.574,41.0,29.0,181.0,195.0,222.0,192.0
3,2019-03-27 17:09:00,74.033,6.426,40.9,28.9,179.0,194.0,220.0,192.0
4,2019-03-27 17:13:00,73.080,5.825,40.9,28.1,181.0,195.0,222.0,193.0


In [8]:
X_test = df_test[["temp","humidity","no2op1","no2op2","o3op1","o3op2"]]
X_test.head()

,temp,humidity,no2op1,no2op2,o3op1,o3op2
0,41.2,28.9,179.0,194.0,220.0,192.0
1,41.1,28.8,181.0,196.0,222.0,192.0
2,41.0,29.0,181.0,195.0,222.0,192.0
3,40.9,28.9,179.0,194.0,220.0,192.0
4,40.9,28.1,181.0,195.0,222.0,193.0


In [9]:
y_test = df_test[["OZONE","NO2"]]
y_test.head()

,OZONE,NO2
0,71.327,8.801
1,72.317,5.536
2,74.440,4.574
3,74.033,6.426
4,73.080,5.825


In [10]:
regr_multirf.score(X_test,y_test)

0.9786440087924173

In [11]:
y_pred = regr_multirf.predict(X_test)

In [12]:
y_pred

array([[71.93115   ,  8.56268385],
       [72.48282   ,  5.82051   ],
       [73.90694   ,  5.30195   ],
       ...,
       [ 9.34205   , 16.2949    ],
       [ 9.90174   , 17.54669   ],
       [ 9.75326   , 16.0868    ]])

In [13]:
mae_o3 = 0
for i in range(y_pred.shape[0]):
    mae_o3 += abs(y_pred[i][0] - y_test["OZONE"][i])
mae_o3/y_pred.shape[0]

1.4326853585940214

In [14]:
mae_no2 = 0
for i in range(y_pred.shape[0]):
    mae_no2 += abs(y_pred[i][1] - y_test["NO2"][i])
mae_no2/y_pred.shape[0]

0.9618431928727624

In [15]:
import pickle

In [16]:
with open('model.pkl','wb') as f:
    pickle.dump(regr_multirf,f)